In [7]:
import cv2
import numpy as np

from sklearn.metrics import pairwise

In [8]:
background = None
accumalated_weight = 0.5
roi_top = 20
roi_bottom = 300
roi_right = 300
roi_left = 600

In [9]:
def calc_bg(frame,accumalated_weight):
    global background
    
    if background is None:
        background = frame.copy().astype('float')
        return None
    cv2.accumulateWeighted(frame,background,accumalated_weight)

In [10]:
def segment(frame,threshold =25):
    diff = cv2.absdiff(background.astype('uint8'),frame)
    
    ret,thresh = cv2.threshold(diff,threshold,255,cv2.THRESH_BINARY)
    
    image,contours,heirarchy = cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours)  == 0:
        return None
    else:
        hand = max(contours,key = cv2.contourArea)
        
        return (thresh,hand)

In [11]:
def count_finger(thresh,hand):
    conv_hull = cv2.convexHull(hand)
    
    top = tuple(conv_hull[conv_hull[:,:,1].argmin()][0])
    bottom = tuple(conv_hull[conv_hull[:,:,1].argmax()][0])
    left = tuple(conv_hull[conv_hull[:,:,0].argmin()][0])
    right = tuple(conv_hull[conv_hull[:,:,0].argmax()][0])
    
    cx = (left[0] + right[0])//2
    cy = (left[1] + right[1])//2
    
    dist = pairwise.euclidean_distances([(cx,cy)],Y = [left,right,top,bottom])[0]
    max_dist = dist.max()
    radius = int(0.7*max_dist)
    circ = 2*np.pi*radius

    roi = np.zeros(thresh.shape[:2],dtype = 'uint8')
    
    cv2.circle(roi,(cx,cy),radius,255,10)
    roi = cv2.bitwise_and(thresh,thresh,mask = roi)
    image,contours,heirarchy = cv2.findContours(roi.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    
    count = 0
    
    for cnt in contours:
        (x,y,w,h) = cv2.boundingRect(cnt)
        
        out_wr = ((cy+(cy*0.25)) > (y+h))
        lim = ((circ*0.25) > cnt.shape[0])
        
        if out_wr and lim:
            count = count+1
            
    return count
    

In [13]:
cam = cv2.VideoCapture(0)

nframes = 0

while True:
    ret,frame = cam.read()
    
    frame_copy = frame.copy()
    
    roi = frame[roi_top:roi_bottom,roi_right:roi_left]
    gray = cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray,(7,7),0)
    
    if nframes <60:
        calc_bg(gray,accumalated_weight)
        
        if nframes<=59:
            cv2.putText(frame_copy,'LOADINGG',(200,200),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
            cv2.imshow('Finger Count',frame_copy)
    else:
        hand = segment(gray)
        
        if hand is not None:
            thresh,hand_seg = hand
            
            cv2.drawContours(frame_copy,[hand_seg+(roi_right,roi_top)],-1,(255,0,0),5)
            
            fingers = count_finger(thresh,hand_seg)
            
            cv2.putText(frame_copy,str(fingers),(70,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
            
            cv2.imshow('Thresholded',thresh)
        
    cv2.rectangle(frame_copy,(roi_left,roi_top),(roi_right,roi_bottom),(0,0,255),5)
    
    nframes = nframes+1
    
    cv2.imshow('Counter',frame_copy)
    
    k = cv2.waitKey(1) & 0xFF
    
    if k ==27:
        break
        
cam.release()
cv2.destroyAllWindows()